<a href="https://colab.research.google.com/github/ayagup/stablediffusion/blob/main/i2v.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

maygup123_images_path = kagglehub.dataset_download('maygup123/images')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install

INFO: pip is looking at multiple versions of opencv-python to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 98.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
"""
Simple Image-to-Video Example
Quick test for image-to-video generation
"""

import torch
from diffusers import StableVideoDiffusionPipeline
from diffusers.utils import load_image, export_to_video
from PIL import Image
import os
import gc

# Suppress warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Memory optimization
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'


def simple_image_to_video(
    image_path: str,
    output_path: str = "output_video.mp4",
    num_frames: int = 14,  # Reduced from 25 for memory efficiency
    motion_strength: int = 127,
):
    """
    Convert an image to video using Stable Video Diffusion

    Args:
        image_path: Path to input image (local file or URL)
        output_path: Path to save output video
        num_frames: Number of frames (14 or 25, must be exact - 14 recommended for T4 GPUs)
        motion_strength: Motion strength (0-255, higher = more motion)
    """

    # Clear GPU memory from any previous runs
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        gc.collect()
        print("🧹 Cleared GPU memory cache\n")

    # Check GPU
    device = "cuda" if torch.cuda.is_available() else "cpu"
    num_gpus = torch.cuda.device_count() if device == "cuda" else 0

    print(f"\n{'='*60}")
    print("Simple Image-to-Video Generation")
    print(f"{'='*60}")
    print(f"Device: {device}")
    if num_gpus > 0:
        print(f"GPUs: {num_gpus}")
        for i in range(num_gpus):
            print(f"  GPU {i}: {torch.cuda.get_device_name(i)}")
    print(f"{'='*60}\n")

    # Load image
    print(f"Loading image: {image_path}")
    if image_path.startswith('http://') or image_path.startswith('https://'):
        image = load_image(image_path)
    else:
        image = Image.open(image_path)

    # Resize to 576x576 (recommended for SVD)
    image = image.resize((576, 576), Image.LANCZOS)
    print(f"Image size: {image.size[0]}x{image.size[1]}")

    # Validate num_frames (SVD only supports 14 or 25)
    if num_frames not in [14, 25]:
        print(f"⚠️  Warning: num_frames must be 14 or 25. Using 14 (memory efficient)")
        num_frames = 14

    # Load model
    print(f"\nLoading Stable Video Diffusion model...")
    print("(This will download ~7GB on first run)\n")

    pipe = StableVideoDiffusionPipeline.from_pretrained(
        "stabilityai/stable-video-diffusion-img2vid",
        torch_dtype=torch.float16 if device == "cuda" else torch.float32,
        variant="fp16" if device == "cuda" else None,
    )

    # Optimize for GPU - ALWAYS use CPU offloading for memory efficiency
    if device == "cuda":
        print(f"🚀 Memory-optimized mode (CPU offloading)")
        if num_gpus > 1:
            print(f"   Distributing across {num_gpus} GPUs\n")
        pipe.enable_model_cpu_offload()
    else:
        pipe = pipe.to(device)

    print("Generating video...")
    print(f"  Frames: {num_frames}")
    print(f"  Motion strength: {motion_strength}")
    print(f"  Decode chunk size: 2 (memory efficient)")
    print(f"  This will take 3-5 minutes...\n")

    # Generate with aggressive memory optimization
    frames = pipe(
        image=image,
        num_frames=num_frames,
        motion_bucket_id=motion_strength,
        decode_chunk_size=2,  # Reduced from 8 for lower memory usage
    ).frames[0]

    # Save
    export_to_video(frames, output_path, fps=7)

    print(f"\n✅ Success!")
    print(f"Video saved to: {output_path}")
    print(f"Frames: {len(frames)}")
    print(f"Duration: ~{len(frames)/7:.1f} seconds\n")

    return frames


if __name__ == "__main__":
    import argparse

    parser = argparse.ArgumentParser(description='Simple Image-to-Video Generation')
    parser.add_argument('--image', type=str, required=True,
                        help='Path to input image')
    parser.add_argument('--output', type=str, default='output_video.mp4',
                        help='Output video path')
    parser.add_argument('--frames', type=int, default=14,
                        help='Number of frames (14 or 25, 14 recommended for T4 GPUs)')
    parser.add_argument('--motion', type=int, default=127,
                        help='Motion strength (0-255)')

    args = parser.parse_args()

    try:
        simple_image_to_video(
            image_path=args.image,
            output_path=args.output,
            num_frames=args.frames,
            motion_strength=args.motion,
        )
    except Exception as e:
        print(f"\n❌ Error: {e}")
        import traceback
        traceback.print_exc()


2025-10-18 10:53:34.344266: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760784814.589771      37 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760784814.659783      37 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
try:
    simple_image_to_video(
        image_path='/kaggle/input/images/smiling-woman-with-crossed-arms-isolated-white-wall.jpg',
        output_path='/kaggle/working/output_video_i2v.mp4',
        num_frames=160,
        motion_strength=127,
    )
except Exception as e:
    print(f"\n❌ Error: {e}")
    import traceback
    traceback.print_exc()



Simple Image-to-Video Generation
Device: cuda
GPUs: 2
  GPU 0: Tesla T4
  GPU 1: Tesla T4

Loading image: /kaggle/input/images/smiling-woman-with-crossed-arms-isolated-white-wall.jpg
Image size: 576x576

Loading Stable Video Diffusion model...
(This will download ~7GB on first run)



Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

🚀 Using 2 GPUs

Generating video...
  Frames: 160
  Motion strength: 127
  This will take 3-5 minutes...



  0%|          | 0/25 [00:00<?, ?it/s]


❌ Error: CUDA out of memory. Tried to allocate 1.76 GiB. GPU 0 has a total capacity of 14.74 GiB of which 1.20 GiB is free. Process 3733 has 13.54 GiB memory in use. Of the allocated memory 13.23 GiB is allocated by PyTorch, and 180.16 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)


Traceback (most recent call last):
  File "/tmp/ipykernel_37/3028529430.py", line 2, in <cell line: 0>
    simple_image_to_video(
  File "/tmp/ipykernel_37/2200763915.py", line 85, in simple_image_to_video
    frames = pipe(
             ^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/_contextlib.py", line 116, in decorate_context
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/diffusers/pipelines/stable_video_diffusion/pipeline_stable_video_diffusion.py", line 584, in __call__
    noise_pred = self.unet(
                 ^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py", line 1739, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py", line 1750, in _call_impl
    return forward_call(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^